Lors du déclage vers le prochain codon, ajouter la valeur de la nouvelle lettre et soustraire l'ancienne

In [4]:
# import numba
from numba import njit

TypeError: create_target_machine() got an unexpected keyword argument 'jitdebug'

In [2]:
import numpy as np

def combiRec4(n,ensemble):
    if n==1:
        return [elem for elem in ensemble]
    else:
        chaines = combiRec4(n-1,ensemble)
        result = [chaine+elem for elem in ensemble for chaine in chaines]
    return result

In [2]:
tableauCombi = combiRec4(3,'ACGT')
print(tableauCombi)

tabCompteur = np.zeros(len(tableauCombi))
dico = {'A':0b00,'C':0b01,'G':0b10,'T':0b11}
s = 'ACGTACGCGCG'

['AAA', 'CAA', 'GAA', 'TAA', 'ACA', 'CCA', 'GCA', 'TCA', 'AGA', 'CGA', 'GGA', 'TGA', 'ATA', 'CTA', 'GTA', 'TTA', 'AAC', 'CAC', 'GAC', 'TAC', 'ACC', 'CCC', 'GCC', 'TCC', 'AGC', 'CGC', 'GGC', 'TGC', 'ATC', 'CTC', 'GTC', 'TTC', 'AAG', 'CAG', 'GAG', 'TAG', 'ACG', 'CCG', 'GCG', 'TCG', 'AGG', 'CGG', 'GGG', 'TGG', 'ATG', 'CTG', 'GTG', 'TTG', 'AAT', 'CAT', 'GAT', 'TAT', 'ACT', 'CCT', 'GCT', 'TCT', 'AGT', 'CGT', 'GGT', 'TGT', 'ATT', 'CTT', 'GTT', 'TTT']


In [3]:
#Initialisation
value = (dico[s[0]])+(dico[s[1]]<<2)+(dico[s[2]]<<4)
tabCompteur[value]+=1

for i in range(3,len(s)):
    value = (value>>2)+(dico[s[i]]<<4) #Shift à droite puis ajout de la lettre suivante à gauche
    tabCompteur[value]+=1

In [4]:
resultat = [[tableauCombi[i],tabCompteur[i]] for i in range(len(tabCompteur))]
print(resultat)

[['AAA', 0.0], ['CAA', 0.0], ['GAA', 0.0], ['TAA', 0.0], ['ACA', 0.0], ['CCA', 0.0], ['GCA', 0.0], ['TCA', 0.0], ['AGA', 0.0], ['CGA', 0.0], ['GGA', 0.0], ['TGA', 0.0], ['ATA', 0.0], ['CTA', 0.0], ['GTA', 1.0], ['TTA', 0.0], ['AAC', 0.0], ['CAC', 0.0], ['GAC', 0.0], ['TAC', 1.0], ['ACC', 0.0], ['CCC', 0.0], ['GCC', 0.0], ['TCC', 0.0], ['AGC', 0.0], ['CGC', 2.0], ['GGC', 0.0], ['TGC', 0.0], ['ATC', 0.0], ['CTC', 0.0], ['GTC', 0.0], ['TTC', 0.0], ['AAG', 0.0], ['CAG', 0.0], ['GAG', 0.0], ['TAG', 0.0], ['ACG', 2.0], ['CCG', 0.0], ['GCG', 2.0], ['TCG', 0.0], ['AGG', 0.0], ['CGG', 0.0], ['GGG', 0.0], ['TGG', 0.0], ['ATG', 0.0], ['CTG', 0.0], ['GTG', 0.0], ['TTG', 0.0], ['AAT', 0.0], ['CAT', 0.0], ['GAT', 0.0], ['TAT', 0.0], ['ACT', 0.0], ['CCT', 0.0], ['GCT', 0.0], ['TCT', 0.0], ['AGT', 0.0], ['CGT', 1.0], ['GGT', 0.0], ['TGT', 0.0], ['ATT', 0.0], ['CTT', 0.0], ['GTT', 0.0], ['TTT', 0.0]]


Sylvain

In [3]:
import traceback
from Bio import SeqRecord, SeqIO
from tqdm.notebook import tqdm

In [4]:
nucleotides = "ACGT"

def kmers_dic(n, choice=nucleotides):
    return {a: 0.0 for a in combinaisons(choice, n)}

def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [f"{a}{n}" for a in combinaisons(combi, n-1) for n in instances]

def seq_count_kmer(seq, kmer_count=None, k=4, ignore_N=True):
    """ Count all kmers, ignore kmers with N or other undecided nucleotides
        seq: string nucleotide input
        kmer_count: dict with all combinations of nucleotides, initialized with zeros (kmer_template["AAAA"] = 0, kmer_count["AAAC"] = 0...)
        the new string hashing behaviour, BiopythonWarning: Using str(seq) to use the new behaviour
    """
    if kmer_count is None:
        kmer_count = kmers_dic(k)
    wrong_base = "N"*k
    kmer_count[wrong_base] = 0.0

    try:
        for kmer in seq_to_window(str(seq), k):
            try:
                kmer_count[kmer] += 1
            except:
                kmer_count[wrong_base] += 1

        if ignore_N:
            kmer_count.pop(wrong_base)
        return kmer_count
    except Exception as e:
        print("type error: " + str(e))
        print(traceback.format_exc())
        return kmer_count

In [20]:
%%timeit
trash = kmers_dic(3)

12.2 µs ± 89.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
kmertest = kmers_dic(3)

In [19]:
%%timeit
trash = kmertest.copy()

1.3 µs ± 9.32 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
def seq_to_window(seq, window_size=3):
    """ Return a sliding window from a string """
    for i in range(len(seq) - window_size + 1):
        yield seq[i:i+window_size]

In [6]:
path_fastq = "/mnt/5TB/reads-fastq/PacBio/communities-silico/silico-3-euka-bac-10k-reads.fastq"

Evaluation

In [10]:
# kmer_dict = kmers_dic(3)
for record in tqdm(SeqIO.parse(path_fastq, "fastq")):
    kmer_dict = kmers_dic(3)
    seq_count_kmer(str(record.seq), None, 3, ignore_N=True)

ghighi

In [12]:
#Copié collé exact de ce qu'il y a au dessus
tabCompteur_vide = np.zeros(len(tableauCombi))
dico = {'A':0b00,'C':0b01,'G':0b10,'T':0b11}
def comptageCodons(chaine,tableauCombi):
    tabCompteur = tabCompteur_vide.copy()
    value = (dico[chaine[0]])+(dico[chaine[1]]<<2)+(dico[chaine[2]]<<4)
    tabCompteur[value]+=1
    for i in range(3,len(chaine)):
        value = (value>>2)+(dico[chaine[i]]<<4) #Shift à droite puis ajout de la lettre suivante à gauche
        tabCompteur[value]+=1
    
    return tabCompteur

In [13]:
tableauCombi = combiRec4(3,'ACGT')
for record in tqdm(SeqIO.parse(path_fastq, "fastq")):
    comptageCodons(str(record.seq),tableauCombi)

In [ ]:
def char_to_value(char):
    if char is "A": return 0b00
    elif char is "C": return 0b01
    elif char is "G": return 0b10
    else: return 0b11

In [8]:
def char_to_value(char):
    # One line if ?
    if char is "A": return 0b00
    elif char is "C": return 0b01
    elif char is "G": return 0b10
    else: return 0b11

In [ ]:
dico = {'A':0b00, 'C':0b01, 'G':0b10, 'T':0b11}
tabCompteur_vide = np.zeros(len(tableauCombi))

In [ ]:
@njit
def comptageCodons_dico(chaine,tableauCombi):
    tabCompteur = tabCompteur_vide.copy()
    value = (dico[chaine[0]])+(dico[chaine[1]]<<2)+(dico[chaine[2]]<<4)
    for i in range(3,len(chaine)):
        value = (value>>2)+(dico[chaine[i]]<<4) #Shift à droite puis ajout de la lettre suivante à gauche
        tabCompteur[value]+=1
    return tabCompteur

In [ ]:
@njit
def comptageCodons_ifs(chaine,tableauCombi):
    tabCompteur = tabCompteur_vide.copy()
    value = char_to_value(chaine[0]) + (char_to_value(chaine[1])<<2) + (char_to_value(chaine[2])<<4)
    tabCompteur[value] += 1
    for i in range(3,len(chaine)):
        value = (value>>2) + (char_to_value(chaine[i])<<4) #Shift à droite puis ajout de la lettre suivante à gauche
        tabCompteur[value]+=1
    return tabCompteur

In [ ]:
%%timeit
comptageCodons_dico(seq)

In [67]:
tableauCombi = combiRec4(3,'ACGT')

In [ ]:
comptageCodons_dico(seq, tableauCombi)

In [ ]:
comptageCodons_ifs(seq, tableauCombi)

In [63]:
%%timeit
comptageCodons_dico(seq, tableauCombi)

1.18 ms ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [64]:
%%timeit
comptageCodons_ifs(seq, tableauCombi)

1.3 ms ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
char = "A"

In [40]:
%%timeit
char == "A"

45.2 ns ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [41]:
%%timeit
char is "A"

22.2 ns ± 0.0761 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [42]:
%%timeit
char == "B"

33.7 ns ± 0.219 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [43]:
%%timeit
char is "B"

22.4 ns ± 0.0545 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [33]:
record.seq

Seq('AGCGTGACTCATCAGAACCATTGGGCGTGCGCCCGTAACAGTGAAGGTCCATGT...GTT', SingleLetterAlphabet())

In [31]:
seq = str(record.seq)

In [30]:
%%timeit
record.seq[3:6]

738 ns ± 0.928 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
%%timeit
seq[3:6]

70 ns ± 0.0824 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [25]:
!ls -alh /mnt/5TB/reads-fastq/PacBio/communities-silico/silico-3-euka-bac-10k-reads.fastq

-rw-rw-r-- 1 ubuntu ubuntu 219M Apr  7 12:15 /mnt/5TB/reads-fastq/PacBio/communities-silico/silico-3-euka-bac-10k-reads.fastq


In [19]:
test = 5

In [24]:
test >> 2

1

In [9]:
#Le comptage est plus rapide :'D
tableauCombi = combiRec4(3,'ACGT')
comptageCodons(chaine,tableauCombi)

array([78124., 77497., 78000., 77954., 78308., 77923., 78392., 78224.,
       78368., 77959., 78238., 78181., 78091., 78083., 78016., 78017.,
       77777., 78728., 78215., 78152., 78090., 78152., 78071., 77767.,
       78028., 78294., 78045., 78032., 78253., 77904., 78369., 77845.,
       77666., 78427., 78069., 78360., 78120., 77806., 78010., 78109.,
       77951., 77942., 78433., 78599., 78132., 78242., 78267., 78371.,
       78008., 78195., 78462., 77741., 78354., 78199., 77926., 78271.,
       78175., 77850., 78209., 78200., 77930., 78521., 77782., 78574.])

In [8]:
#L'affichage de la chaine est aussi lente haha
chaine[:200]

'TCCGGTACACTGATATGTTACGAACATATCACGACATAGATTGCGCTTGCGGTAAGGCCCCTCTCTCGGTTCCTTAATCTCGTAGACGCATGGAAGTCTAAAGTAATCGGCTACCCCCTCCCATTTTATACGAATTTGTCAGGCCCACAGTGCCTCCGACTTATGGTTTTTGTGCTCATCTAAATCTAAGAATCTATTCG'